In [ ]:
!pip install openai
!pip install docx
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=a4e810a10061d5ecf190b06f79c4a21698a7d69197d7b60397e9ffe0df16bd5b
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from google.colab import userdata
import time
import openpyxl
import requests as req
import os
import time
from types import NoneType
import ast
import re
import json
import pandas as pd
from docx import Document

In [ ]:
strategies_list = [
    "organizing and transforming", "goal setting and planning", "seeking information",
    "keeping records and monitoring", "environmental structuring", "self-consequences", "rehearsing and memorizing",
    "seeking social assistance", "reviewing records", "other"
]

strategies_list = list(map(lambda x: x.lower(), strategies_list))

strategies_abbr = [
    "org.", "planning", "seek.info",
    "keep.rec.", "env.", "self.cons.", "reaherse.",
    "seek.social", "review.rec.", "others"
]

In [ ]:
system_prompt = """
I would like to invite you to participate as a linguistic researcher in my research project. The objective of this project is to analyse written reflections from students concerning their self-regulated learning (SRL) strategies. I have asked the students to reflect on the learning strategies they employed in their previous work. My main goal is to determine whether students mention self-regulated learning strategies in their assessments and, if they do, what specific SRL strategies they mention. It's important to note that a single sentence in a student's reflection may reflect multiple SRL strategies, indicating the complexity and interconnectedness of self-regulated learning processes.
"""

In [ ]:
def docx_to_string(file_path):
    # Open the .docx file
    doc = Document(file_path)
    # Read and concatenate all the text from the document
    full_text = [paragraph.text for paragraph in doc.paragraphs]
    return '\n'.join(full_text)

In [ ]:
user_prompt = docx_to_string('/content/sourcedata/prompt-1-shot.docx')

In [ ]:
def is_numeric(val):
    """Check if the value is numeric."""
    try:
        int(val)
        return True
    except ValueError:
        return False

In [ ]:
def getOpenAIResp(sys, user, modelid):

  resp = client.chat.completions.create(
    model = modelid,
    messages=[
       {"role": "system", "content": sys},
       {"role": "user", "content": user}
    ]
  )
  return resp.choices[0].message.content

In [ ]:
def call_openai_api(system_prompt, user_prompt, model):
    c3l_openai_key = 'PASTE YOUR OPENAI API KEY HERE'
    client = OpenAI(api_key=c3l_openai_key)

    completion = client.chat.completions.create(
      model=model,
      temperature = 0,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
      ]
    )

    result = completion.choices[0].message.content
    time.sleep(1)  # Simulate network delay
    return result

In [ ]:
def get_strategies(api_responses):

  # Escape special characters in strategy names and create a regex pattern
  escaped_strategies = [re.escape(strategy) for strategy in strategies_list]
  pattern = r'\b(' + '|'.join(escaped_strategies) + r')\b'

  # Find all matches in the text
  found_strategies = re.findall(pattern, api_responses, re.IGNORECASE)

  return found_strategies


In [ ]:
def print_reasoning_excel(rdict):
  # Convert the dictionary to a pandas DataFrame
  df = pd.DataFrame.from_dict(rdict, orient='index')

  # Export the DataFrame to an Excel file
  df.to_excel('reasoning.xlsx', index=False)

In [ ]:
def print_reflection_coding_excel(rdict, num_runs):
    """
    Generates detailed and summary Excel files from reflection coding data.

    Args:
        reflection_dict (dict): Dictionary containing reflection coding data.
        num_runs (int): Number of runs to consider for calculating majority votes.

    Returns:
        None
    """
    # Lists to hold row data for detailed and summary DataFrames
    details_rows = []
    summary_rows = []

    # Iterate over the reflection coding data
    for row_number, strategies in rdict.items():
        row_data = {} # Dictionary to hold detailed row data
        summary_data = {} # Dictionary to hold summary row data

        # Iterate over each strategy in the row
        for strategy, runs in strategies.items():
            votes = [] # List to keep track of votes for the strategy

            # Iterate over each run for the current strategy
            for run_number, value in runs.items():
                # Generate a column name for the strategy and run
                column_name = f"{strategy}_run{run_number}"
                row_data[column_name] = value  # Assign the value (0 or 1)
                votes.append(value)

            # Calculate majority vote for the strategy
            if votes.count(1) > num_runs / 2:
                summary_data[strategy] = 1
            else:
                summary_data[strategy] = 0

        # Add the detailed and summary row data to their respective lists
        details_rows.append(row_data)
        summary_rows.append(summary_data)

    # Create DataFrame for detailed data and save to Excel
    df = pd.DataFrame(details_rows)
    df.index += 1
    df.to_excel('strategies_detailed.xlsx', engine='openpyxl')

    # Convert the summary data into another DataFrame
    summary_df = pd.DataFrame(summary_rows)
    summary_df.index += 1
    summary_df.to_excel('strategies_summary.xlsx', engine='openpyxl')

In [ ]:
def print_coding(rdict, num_rows, num_iter):
  """
  Processes reflection data and prints a coding dictionary for analysis.

  Args:
      reflection_dict (dict): Dictionary containing reflection data.
      num_rows (int): Number of rows to consider for processing.
      num_iter (int): Number of iterations for coding.

  Returns:
      None
  """

  # Dictionary to store the final coded reflections
  reflection_coding_dict = {}

  # Iterate over the reflection dictionary
  for k, v in rdict.items():

    # Initialize a temporary coding dictionary for the current row
    temp_coding_dict = {
      row: {
          s: {i: 0 for i in range(1, num_iter + 1)}
          for s in strategies_abbr
      } for row in range(k, num_rows + k)
    }

    # Iterate over each API call response
    for run, codes_text in v.items():
      # Extract strategies from the response text
      for strat in get_strategies(codes_text):
        # Find the corresponding strategy ID
        strategy_id = strategies_list.index(strat.lower())

        # Set the coding value to 1 for the identified strategy
        temp_coding_dict[k][strategies_abbr[strategy_id]][run] = 1

    # Store the coded data for the current row
    reflection_coding_dict[k] = temp_coding_dict[k]

  # Generate an Excel file with the coded reflection data
  print_reflection_coding_excel(reflection_coding_dict, num_iter)

In [ ]:
def process_excel_with_openai(excel_file_path, num_api_calls, model):
    """
    Processes an Excel file and makes multiple API calls for each row of data.

    Args:
        excel_file_path (str): Path to the Excel file.
        num_api_calls (int): Number of times to call the API for each row.
        model (str): The model to be used for API calls. Some of the options
        include gpt-4-0125-preview, gpt-3.5-turbo-0125, or gpt-4-turbo-preview

    Returns:
        None
    """

    # Load the workbook and select the active worksheet
    workbook = openpyxl.load_workbook(excel_file_path)
    worksheet = workbook.active  # Assuming the data is in the active sheet

    # Dictionary to store the responses for each row
    reasoning_dict = {}
    count_of_reflections = 0

    # Iterate over rows starting from the second row (assuming first row is header)
    for row in worksheet.iter_rows(min_row=2):
        id_cell, text_cell = row[0], row[1]

        # Check if the ID cell contains a numeric value
        if is_numeric(val=id_cell.value):
          count_of_reflections += 1
          print(f"Processing ID: {id_cell.value} | Text: {text_cell.value}")

          # Dictionary to store responses for multiple API calls
          bootstrap_dict = {}
          for run in range(num_api_calls):
            # Prepare the user prompt by appending the text from the cell
            user_p = user_prompt + '\\n"' + text_cell.value

            # Call the OpenAI API
            response = call_openai_api(system_prompt=system_prompt,
                                       user_prompt=user_p,
                                       model = model)

            # Store the response in the bootstrap dictionary
            bootstrap_dict[run+1] = response

          # Store the bootstrap dictionary in the reasoning dictionary
          reasoning_dict[id_cell.value] = bootstrap_dict

    # Print the results
    print_reasoning_excel(reasoning_dict)
    print_coding(reasoning_dict, count_of_reflections, num_api_calls)

# Setup the parameters and invoke the function
excel_file_path = '/content/sourcedata/10-examples.xlsx'
num_api_calls = 3  # Number of times you want to call the API for each valid row
model = 'gpt-4o'

process_excel_with_openai(excel_file_path,
                          num_api_calls,
                          model=model)




Processing ID: 11 | Text: Changed up a few things but mainly stuck with the plan, didn't end up with enough time due to other circumstances. 
Processing ID: 14 | Text: To compare the two articles, I first critically read and analysed each of them. Upon comprehension, it became evident that the article I chose myself was denser in academic content in contrast to the casually written recount in the Geist & Jung (2022) article. This became my focus regarding comparison of the two articles. I used the method of Boolean operators in order to locate a related article on the UniSA library data base. I searched the term ‘early childhood education AND garden-based learning’, which led me to finding the Murakami, Russell & Manfra (2018) article. This article complemented the Geist & Jung article in that it was based on the concept of garden-based learning in early childhood education, but also contained a case study analysis.
Processing ID: 15 | Text: My initial strategy was to locate a recent a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')